In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions.categorical import Categorical
import matplotlib.pyplot as plt

In [3]:
import sys
sys.argv = [""]

In [13]:
from cart_pole.utils import *
from cart_pole.agent import Agent
from torch.utils.tensorboard import SummaryWriter
from cart_pole.environment import *
import pdb

In [14]:
args = parse_args(seconds_since_epoch=1764177877)
writer = init_tracking(args)

In [ ]:
rng = init_rngs(args)
seeds = [int(rng.uniform(0, 1E9)) for _ in range(args.num_envs)]

envs = make_sync_vector_env(args)
storage = Storage(args, envs)

agent = Agent(envs).to(args.device)

optimizer = optim.Adam(agent.parameters(),
                       lr=args.learning_rate,
                       eps=1e-5)

next_obs = torch.Tensor(envs.reset(seed=seeds)[0]).to(args.device)

next_terminated =\
    torch.Tensor(np.array([False] * args.num_envs)).to(args.device)

next_truncated =\
    torch.Tensor(np.array([False] * args.num_envs)).to(args.device)

info = {}

num_updates = args.total_timesteps // args.batch_size
storage.reset(envs)

for _ in range(args.num_steps):

    storage.store_state_info(next_obs,
                             next_terminated,
                             next_truncated,
                             info)

    with torch.no_grad():

        actions, logprobs, _, values = agent.get_action_and_value(next_obs)

        storage.store_action_info(values.flatten(),
                                  actions,
                                  logprobs)

        next_obs, rewards, terminated, truncated, info =\
            envs.step(actions.cpu().numpy())

        rewards = torch.tensor(rewards).to(args.device).view(-1)
        storage.store_rewards(rewards)
        
        next_obs = torch.Tensor(next_obs).to(args.device)
        next_terminated = torch.Tensor(terminated).to(args.device)
        next_truncated = torch.Tensor(truncated).to(args.device)

--Return--
None
> /tmp/ipykernel_777/2282553160.py(39)store_state_info()
     37                 torch.tensor(epr_values).to(self.device)
     38 
---> 39             pdb.set_trace()
     40 
     41     def store_action_info(self,



ipdb>  self.rewards[:10,:]


tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [0., 0., 0., 0.]], device='cuda:0')


ipdb>  self.episodic_rewards[:10,:]


tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 9., 0., 0.]], device='cuda:0')


ipdb>  self.truncated[:10,:]


tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], device='cuda:0')


ipdb>  self.terminated[:10,:]


tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 1., 0., 0.]], device='cuda:0')


In [ ]:
terminated = storage.terminated.cpu()
plt.plot(terminated[:,0])
plt.plot(terminated[:,1])
plt.plot(terminated[:,2])

In [ ]:
terminated